In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_datareader as pdr
import datetime
import numpy as np

In [2]:
start = datetime.datetime(2010, 1, 1)
end = datetime.datetime(2020,12,31)

In [60]:
# 삼성전자 10년치 일 데이터 - 하나의 일 데이터를 하나의 tick data로 생각
# close price를 p_t 라고 생각
df = pdr.get_data_yahoo('005930.KS', start, end)
tick = df[['Volume', 'Close']]
tick

,Volume,Close
Date,,
2010-01-04,11950800.0,16180.0
2010-01-05,27925850.0,16440.0
2010-01-06,22948850.0,16820.0
2010-01-07,22107950.0,16260.0
2010-01-08,14777550.0,16420.0
...,...,...
2020-12-23,19411326.0,73900.0
2020-12-24,32502870.0,77800.0
2020-12-28,40085044.0,78700.0


In [61]:
# b_t(tick rule)
def tick_rule(df):
    diff = df['Close'] - df['Close'].shift(1)
    b_t = np.abs(diff) / diff
    return b_t

In [84]:
# 첫 번째 tib(tick imbalance bar) 정하기위해서는 최초의 값들을 정해줘야 함(이전 bar size, 이전 tick rule 값)
def tick_imbalance_bar(df, initial_expected_bar_size = 3, initial_expected_tick_rule = 1):
    
    
    i = 0
    bar_dict = dict()
    
    while i < 4:

        if i == 0:
            temp = df
            temp['T'] = np.arange(len(temp)) + 1
            temp['tick_rule'] = tick_rule(tick).fillna(method = 'ffill')
            temp['tick_rule'][0] = 1
            temp['tick_imbalance'] = temp['tick_rule'].cumsum()
            temp['expected_tick_imbalance'] = initial_expected_bar_size * initial_expected_tick_rule
            result = temp[abs(temp['tick_imbalance']) - temp['expected_tick_imbalance'] >=0]
            T_star = result.index[0]
            bar_sample = temp.loc[:T_star, :]

            bar_dict[i] = bar_sample
            i += 1
            df = df.loc[T_star + datetime.timedelta(days = 1):, :]

        else:
            temp = df
            temp['T'] = np.arange(len(temp)) + 1
            temp['tick_rule'] = tick_rule(tick).fillna(method = 'ffill')
            temp['tick_rule'][0] = 1
            temp['tick_imbalance'] = temp['tick_rule'].cumsum()
            estimated_E_T = bar_dict[i-1].ewm(com = 0.5).mean()['T'][-1]
            estimated_b_t = bar_dict[i-1].ewm(com = 0.5).mean()['tick_rule'][-1]
            temp['expected_tick_imbalance'] = estimated_E_T * np.abs(estimated_b_t)
            result = temp[abs(temp['tick_imbalance']) - temp['expected_tick_imbalance'] >=0]
            T_star = result.index[0]
            bar_sample = temp.loc[:T_star, :]

            bar_dict[i] = bar_sample
            i += 1
            df = df.loc[T_star + datetime.timedelta(days = 1):, :]

    return bar_dict

In [86]:
result_bar = tick_imbalance_bar(tick)

c:\users\minguk\desktop\coding\finance_ml\venv\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\minguk\desktop\coding\finance_ml\venv\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\minguk\desktop\coding\finance_ml\venv\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [92]:
result_close = dict()
for i in result_bar.keys():
    key = result_bar[i].index[-1]
    values = result_bar[i]['Close'][-1]
    result_close[key] = values
    
result_close

{Timestamp('2010-01-06 00:00:00'): 16820.0,
 Timestamp('2010-01-15 00:00:00'): 16840.0,
 Timestamp('2010-02-25 00:00:00'): 14720.0,
 Timestamp('2016-06-03 00:00:00'): 27540.0}

In [45]:
x.index[0] + datetime.timedelta(days = 1)

Timestamp('2011-03-03 00:00:00')